In [157]:
import requests, re, urllib, itertools
from bs4 import BeautifulSoup
from bs4.element import Comment
from urllib.parse import urlparse
from collections import Counter, Iterable
from nltk.corpus import wordnet as wn
from gensim.models import KeyedVectors
from anytree import Node
from nltk.tree import Tree
from nltk.chunk.regexp import RegexpParser
from nltk import pos_tag, word_tokenize
from lxml.html import fromstring
from itertools import cycle
import traceback, time
request = 0
def checkAppropriateURL(url):
    parsedURL = urlparse(url)   

    extn = parsedURL.path.split(".")[-1]
    site = parsedURL.netloc.split(".")
    path = parsedURL.path.split("/")[-1]

    if extn in ["pdf","doc","docx","xls","xlsx","ppt","pptx","odt"]:
        return False
    if any(i in site for i in ["pinterest"]):
        return False
    try:
        r = dict(requests.head(url).headers)["Content-Type"].split(";")[0]
    except:
        return False
    if(not r=="text/html"):
        return False
    return True

def scrapeGoogleForAbstracts(query, count):
    global request
    words = query.split()
    q = ("+").join(words)
    print (q)
    query_url_goog = "https://www.google.co.in/search?q=\"" + q + "\""
    
    request += 1
    print ("Making request ", request)
    try:
        data = requests.get(query_url_goog).text
    except Exception as e:
        print (e)
        return
    if "detected unusual traffic" in data.lower():
        print ("Rate limited")
        return
    time.sleep(sleepingTime)
#     print (data)
    soup = BeautifulSoup(data, "html.parser")
    h3Rows = [el for el in soup.find_all("div", {"class":"BNeawe s3v9rd AP7Wnd"}) if el.find("div", {"class":"BNeawe s3v9rd AP7Wnd"})]
    spanRows = [el for el in soup.find_all("div", {"class":"kCrYT"}) if el.find("a")]
    extra = len(spanRows) - len(h3Rows)
    spanRows2 = spanRows[extra:]
    zippedRows = zip(spanRows2,h3Rows)

    urls = []
#     print (list(zippedRows))
    for zippedRow in zippedRows:
        try:
            h3Row = zippedRow[0]
            url = h3Row.find("a")['href']
            if(url.split("?")[0]=="/search"):
                continue
        except (GeneratorExit, KeyboardInterrupt, SystemExit):
            raise
        except:
            continue
        par = urllib.parse.parse_qs(urlparse(url).query)
        try:
            appendingURL = par['q'][0]
        except:
            continue
        if(not checkAppropriateURL(appendingURL)):
            continue
        matchedQuery = query
        allbElems = zippedRow[1].findAll('b')
        for bElem in allbElems:
            if bElem.string[0].isalnum():
                matchedQuery = bElem.string
        urls.append((matchedQuery,appendingURL))
    
    i = 1
    same = 0
    prev = 0
    jj = 0
    exceptiontime = 0
    while(len(urls)<count):
#         print ("Query: ",query)
        tempurl = query_url_goog + "&start=" +str(i*10)
        i+=1

        request += 1
        print ("Making request ", request)
        try:
            data = requests.get(tempurl).text
            exceptiontime = 0
        except Exception as e:
            print (e)
            exceptiontime += 1
            if exceptiontime > 3:
                break
            continue
        if "detected unusual traffic" in data.lower():
            print ("Rate limited")
            return
        time.sleep(sleepingTime)
        soup = BeautifulSoup(data, "lxml")
        h3Rows = [el for el in soup.find_all("div", {"class":"BNeawe s3v9rd AP7Wnd"}) if el.find("div", {"class":"BNeawe s3v9rd AP7Wnd"})]
        spanRows = [el for el in soup.find_all("div", {"class":"kCrYT"}) if el.find("a")]
        extra = len(spanRows) - len(h3Rows)
        spanRows2 = spanRows[extra:]
        zippedRows = zip(spanRows2, h3Rows)
        for (j,zippedRow) in enumerate(zippedRows):
            try:
                h3Row = zippedRow[0]
                url = h3Row.find("a")['href']
            except (GeneratorExit, KeyboardInterrupt, SystemExit):
                raise
            except:
                continue
            par = urllib.parse.parse_qs(urlparse(url).query)
            try:
                appendingURL = par['q'][0]
                if(not checkAppropriateURL(appendingURL)):
                    continue
                matchedQuery = query
                allbElems = zippedRow[1].findAll('b')
                for bElem in allbElems:
                    if bElem.string[0].isalnum():
                        matchedQuery = bElem.string
                urls.append((matchedQuery, appendingURL))
            except (GeneratorExit, KeyboardInterrupt, SystemExit):
                raise
            except:
                continue
        if prev == len(urls):
            same +=1 
            if same>=10:
                same = 0
                break
        prev = len(urls)
    return urls[:count]

In [158]:


def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

def getTextFromURL(url):
    hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
   'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
   'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
   'Accept-Encoding': 'none',
   'Accept-Language': 'en-US,en;q=0.8',
   'Connection': 'keep-alive'}
    html = urllib.request.urlopen(urllib.request.Request(url, headers=hdr))
    inputText = text_from_html(html)
#     print (inputText)
    return inputText

def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:100]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            #Grabbing IP and corresponding PORT
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return proxies


def getAbstracts(query, count):

    urls = scrapeGoogleForAbstracts(query, count)
#     print (urls)
#     print ("lenght",len(urls))
    texts = []
    for (query, url) in urls:
#         print (url)
        try:
            text = getTextFromURL(url)
        except (GeneratorExit, KeyboardInterrupt, SystemExit):
            raise
        except:
            continue
        texts.append((query,url,text.strip()))
    return texts


In [162]:
def processWordSet(wordSet):
    for (i,word) in enumerate(wordSet):
        wordSet[i] = re.sub("^[^a-zA-Z]*|[^a-zA-Z(?<!(s\'))]*$","",word)
    return wordSet

def getHypernymsFromWordnet(word):
    allSynsets = wn.synsets(word)
    multiList = [[[a.name() for a in b.lemmas()] for b in c.hypernyms()]for c in wn.synsets(word)]
    finalList = flatten(multiList)
    return Counter(finalList)

def compareTuple(x,model):
    try:
        sim = model.similarity(x[0],x[1])
        return sim
    except:
        return 0

def initNodesList(wordSet):
    nodesList = []
    for wordNode in wordSet:
        nodesList.append(Node(wordNode))
    return nodesList

def flatten(l):
    for el in l:
        if isinstance(el, Iterable) and not isinstance(el, (str, bytes)):
            yield from flatten(el)
        else:
            yield el

# file = open("pizza.txt")
# text = file.read()
# wordSet = processWordSet(list(set(text.split())))
# wordSet = list(set(filter(None, wordSet)))

wordSet = [l.strip() for l in open("concepts_security-small.txt","r", encoding="utf8").read().split("\n")]

# model = KeyedVectors.load_word2vec_format("~/GoogleNews-vectors-negative300.bin", binary=True)

# # Sort by decreasing similarity
# tuplesList = list(itertools.combinations(wordSet,2))
# sortedList = sorted(tuplesList, key=lambda x : compareTuple(x,model), reverse=True)

clusteredSet = set()
hypernymDict = {}

root = Node("root")
nodesList = initNodesList(wordSet)


In [163]:
def getHearstPatterns(word):
    # Pattern, Instances after/before, One/many instances
    patterns = []
    patterns.append((word + " such as", True, True))
    patterns.append(("such " + word + " as", True, True))
    patterns.append(("and other " + word, False, True))
    patterns.append(("or other " + word, False, True))
    patterns.append((word + ", including", True, True))
    patterns.append((word + ", especially", True, True))
    return patterns

def getInstances(text):
    grammar = """
        PRE:   {<NNS|NNP|NN|NP|JJ|UH>+}
        MID: {<DT|IN|POS|FW|-|NP|NPS|NN|NNS>+}
        INSTANCE:   {(<DT+>)?(<JJ+>)?<PRE>(<MID><PRE>)?}
    """
    chunker = RegexpParser(grammar)
    taggedText = pos_tag(word_tokenize(text))
    textChunks = chunker.parse(taggedText)
    current_chunk = []
    for i in textChunks:
        if (type(i) == Tree and i.label() == "INSTANCE"):
            # print (i.leaves())
            current_chunk.append(" ".join([token for token, pos in i.leaves()]))
    return current_chunk

def extractHypernyms(posList, text, query):
    allInstances = []
    for (st,end) in posList:
        if query[2]:
            if query[1]:
                #Instances after, and exists                
                tempText = text[end+1:]
                rest = re.match(".*?\.",tempText)
                if rest:
                    rest = rest.group()
                else:
                    rest = tempText
                andExists = re.search(" (and|or) ", rest)
                if andExists:
#                     print ("rest:", rest)
                    lastConceptPos = andExists.end()
                    lastConcept = rest[lastConceptPos:]
#                     print ("lastConcept: ", lastConcept)
                    lastConceptExtracted = getInstances(lastConcept.strip())
                    if lastConceptExtracted:
                        allInstances.append(lastConceptExtracted[0])
                    prevConceptsEnd = andExists.start()
                    prevConcepts = rest[:prevConceptsEnd]
                    prevConceptsList = prevConcepts.split(",")
                    prevConceptsList = [a for a in prevConceptsList if a.strip()]
#                     print ("****START")
#                     print ("previous Concepts:",prevConcepts)
                    for concept in prevConceptsList:
                            print (concept.strip(), getInstances(concept.strip()))
                            conceptExtracted = getInstances(concept.strip())
                            if conceptExtracted:
                                allInstances.append(conceptExtracted[-1])
#                     print ("*****DONE")
                else:
                    allConcepts = rest.split(",")
#                     print ("#####HERE ARE ALL CONCEPTS:",allConcepts)
                    for (i,concept) in enumerate(allConcepts):
                            if i==(len(allConcepts)-1):
                                print (concept)
                                conceptExtracted = getInstances(concept.strip())
#                                 print ("1######CONCEPT EXTRACTED:", conceptExtracted)
                                if conceptExtracted:
                                    allInstances.append(conceptExtracted[0])
                            else:
                                conceptExtracted = getInstances(concept.strip())
#                                 print ("######CONCEPT EXTRACTED:", conceptExtracted)
                                if conceptExtracted:
                                    allInstances.append(conceptExtracted[-1])
            else:
                tempText = text[:st]
#                 print ("tempText", tempText)
                try:
                    lastSentPos = [m.end() for m in re.finditer(".*?\.",tempText)][-1]
                    lastSent = tempText[:lastSentPos]
                except:
                    lastSent = tempText
                lastSentList = lastSent.split(",")
                lastSentList = [a for a in lastSentList if a.strip()]
                if len(lastSentList) == 1:
                    conceptExtracted = getInstances(lastSentList[0].strip())
                    if conceptExtracted:
                        allInstances.append(conceptExtracted[-1])
                else:
                    for (i,concept) in enumerate(lastSentList):
                        conceptExtracted = getInstances(concept.strip())
                        if conceptExtracted:
                            if i==(len(lastSentList)-1):
                                allInstances.append(conceptExtracted[0])
                            else:
                                allInstances.append(conceptExtracted[-1])
        else:
            tempText = text[:st]
            try:
                lastSentPos = [m.end() for m in re.finditer(".*?\.",tempText)][-1]
                lastSent = tempText[:lastSentPos]
            except:
                lastSent = tempText
            lastConceptExtracted = getInstances(lastSent.strip())
            if lastConceptExtracted:
                allInstances.append(lastConceptExtracted[-1])
    return allInstances

def getHypernymsFromWWW(word):
    allHypernyms = []
    queriesList = getHearstPatterns(word)
    for query in queriesList:
        texts = getAbstracts(query[0], 5)
#         print (query[0], texts)
        for (q,url,text) in texts:
            startEndPositions = []
            try:
                for m in re.finditer(re.escape("\W\W?".join(q.split("+"))), text, re.I):
                    try:
                        startEndPositions.append((m.start(), m.end()))
                    except:
#                         print ("query:",q,"url:",url)
                        continue
            except:
#                 print ("1","query:",q,"url:",url)
                continue
            allHypernyms.extend(extractHypernyms(startEndPositions, text, query))
    res = Counter(allHypernyms)
    print (res)
    return res

proxies = get_proxies()
proxy_pool = cycle(proxies)
            
def getHypernyms(word):
    
    wordnet = getHypernymsFromWordnet(word)
    www = getHypernymsFromWWW(word)
#     print (www)
    return (wordnet + www)
        

In [141]:
ls = {}
ls["dict"] = Counter([1,2,3]) + Counter([1,3,6])
ls

{'dict': Counter({1: 2, 2: 1, 3: 2, 6: 1})}

In [164]:
sleepingTime = 30
for word in wordSet:
    hypernymDict[word] = getHypernyms(word)
    

Vulnerability+In+Code+such+as
Making request  2
buffer overrun ['buffer overrun']
buffer overrun ['buffer overrun']
buffer overrun ['buffer overrun']
such+Vulnerability+In+Code+as
Making request  3
and+other+Vulnerability+In+Code
Making request  4
or+other+Vulnerability+In+Code
Making request  5
Vulnerability+In+Code,+including
Making request  6
Vulnerability+In+Code,+especially
Making request  7
Counter({'bugs like Heartbleed': 3, 'buffer overrun': 3})
Access+Control+Model+such+as
Making request  8
The Role Based Access Control (RBAC) model [3] that was extended to a new model named context based access control by the introduction of context which is provided by the web service.
such+Access+Control+Model+as
Making request  9
Clark-Biba.
Policy Access Control.
implicit allow.
and+other+Access+Control+Model
Making request  10
Making request  11
Making request  12
Making request  13
Making request  14
Making request  15
Making request  16
Making request  17
Making request  18
Making requ

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


accepting the way of biological data collection.
UI design ['UI design']
user-device interaction design ['user-device interaction design']
data collection method ['data collection method']
authentication protocol design ['authentication protocol design']
Biometric+Authentication+System,+especially
Making request  60
Making request  61
Making request  62
Making request  63
Making request  64
Making request  65
Making request  66
Making request  67
Making request  68
Making request  69
Making request  70
Making request  71
given that it’s going to run on Android ['Android']
Counter({'template security': 1, 'reissuance of biometric template': 1, 'iris authentication compliant smartphone': 1, 'fingerprint authentication device': 1, 'way of biological data collection': 1, 'UI design': 1, 'user-device interaction design': 1, 'data collection method': 1, 'authentication protocol design': 1, 'new Android M OS': 1, 'Android': 1})
Database+Management+System+such+as
Making request  72
 Sybase etc

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


how it securely stores data ['stores data']
enforces complex business requirements ['enforces complex business requirements']
Database+Management+System,+especially
Making request  87
in Grid environment ['Grid environment']
it is important to establish the mechanism of identity authentication ['important', 'mechanism of identity authentication']
Counter({'data types': 2, 'ADO': 2, 'SQL': 2, 'WI': 2, 'Oracle': 1, 'RDBMS': 1, 'Sybase etc': 1, 'ODBMS': 1, 'MySQL Server': 1, 'access': 1, 'free': 1, 'cons of technologies': 1, 'management software from DBMS vendors': 1, 'orchestration': 1, 'governance': 1, 'SQLite': 1, 'tutorial': 1, 'static types': 1, 'other database management systems': 1, 'column with a data type integer': 1, 'values of data type integer': 1, 'dynamic types like in SQLite': 1, 'type of value': 1, 'INTEGER values in SQLite': 1, 'point values': 1, 'UTF-8': 1, 'UTF-16 BE': 1, 'large files': 1, 'column': 1, 'type of data': 1, 'types': 1, 'page on databases': 1, 'homemade dat

...

or+other+Theft+Protection+Software
Making request  279
Theft+Protection+Software,+including
Making request  280


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Theft+Protection+Software,+especially
Making request  281
Counter({'LoJack for Laptops': 1, 'Prey Anti-Theft': 1})
Data+Deletion+Software+such+as
Making request  282
Making request  283
Making request  284
Making request  285
Making request  286
Making request  287
Making request  288
Making request  289
Making request  290
Making request  291
Making request  292
Making request  293
such+Data+Deletion+Software+as
Making request  294
and+other+Data+Deletion+Software
Making request  295
or+other+Data+Deletion+Software
Making request  296
Data+Deletion+Software,+including
Making request  297
Data+Deletion+Software,+especially
Making request  298
Counter()
Secure+Network+Communication+such+as
Making request  299
Microsoft Windows ['Microsoft Windows']
embedded XP ['XP']
ssh.
Microsoft Windows ['Microsoft Windows']
embedded XP ['XP']
Microsoft Windows ['Microsoft Windows']
embedded XP ['XP']
such+Secure+Network+Communication+as
Making request  300
and+other+Secure+Network+Communication
Maki

Making request  339
Making request  340
Making request  341
Making request  342
Making request  343
Making request  344
Making request  345
Making request  346
Making request  347
Making request  348
Making request  349
such+HTML+Script+Injection+as
Making request  350
and+other+HTML+Script+Injection
Making request  351
or+other+HTML+Script+Injection
Making request  352
HTML+Script+Injection,+including
Making request  353
HTML+Script+Injection,+especially
Making request  354
Counter()
Low+Level+Threat+such+as
Making request  355
such+Low+Level+Threat+as
Making request  356
and+other+Low+Level+Threat
Making request  357
Making request  358
Making request  359
Making request  360
Making request  361
Making request  362
Making request  363
Making request  364
Making request  365
Making request  366
Making request  367
Making request  368
or+other+Low+Level+Threat
Making request  369
Making request  370
Making request  371
Making request  372
Making request  373
Making request  374
Making 

TypeError: 'NoneType' object is not iterable